## Initial code, straight from Chat GPT and pairs trading course

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
# Load data for two assets
asset1 = pd.read_csv("asset1.csv", index_col=0)
asset2 = pd.read_csv("asset2.csv", index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'asset1.csv'

In [13]:
# Compute the spread between the two assets
spread = asset1 - asset2

NameError: name 'asset1' is not defined

In [15]:
# Compute the rolling mean and standard deviation for each asset
asset1_mean = asset1.rolling(window=30).mean()
asset1_std = asset1.rolling(window=30).std()
asset2_mean = asset2.rolling(window=30).mean()
asset2_std = asset2.rolling(window=30).std()

NameError: name 'asset1' is not defined

In [17]:
# Compute z-scores for each asset based on its rolling mean and standard deviation
z1 = (asset1 - asset1_mean) / asset1_std
z2 = (asset2 - asset2_mean) / asset2_std

NameError: name 'asset1' is not defined

In [18]:
# Compute the rolling mean and standard deviation of the spread
spread_mean = spread.rolling(window=30).mean()
spread_std = spread.rolling(window=30).std()

NameError: name 'spread' is not defined

In [19]:
# Compute the rolling z-score for the spread
spread_zscore = (spread - spread_mean) / spread_std

NameError: name 'spread' is not defined

In [8]:
# Compute the upper and lower bounds for the spread
upper_bound = spread_mean + 2 * spread_std
lower_bound = spread_mean - 2 * spread_std

NameError: name 'spread_mean' is not defined

In [20]:
# Compute the upper and lower bounds for the spread
long_signal = (spread_mean < -1).astype(int)
short_signal = (spread_mean > 1).astype(int)

NameError: name 'spread_mean' is not defined

In [10]:
# Compute the positions for each asset
asset1_position = -long_signal + short_signal
asset2_position = long_signal - short_signal

NameError: name 'long_signal' is not defined

In [11]:
# Compute the portfolio returns
portfolio_returns = asset1_position.shift(1) * z1 + asset2_position.shift(1) * z2

NameError: name 'asset1_position' is not defined

In [12]:
# Compute the total returns and plot the results
total_returns = portfolio_returns.sum(axis=1)
plt.plot(total_returns.cumsum())
plt.title("Pairs Trading Algorithm")
plt.xlabl("Date")
plt.ylabel("Total Returns")
plt.show()

NameError: name 'portfolio_returns' is not defined